In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS DescEKGCases;

CREATE TABLE DescEKGCases (
  user INT NOT NULL,
  casen VARCHAR(7),
  stage VARCHAR(22),
  answer VARCHAR(200),
  points DECIMAL(3,1),
  confidence INT,
  time DECIMAL(8,2),
  ctype VARCHAR(2),
  PRIMARY KEY(user, casen, stage)
) AS SELECT *
  FROM CSVREAD('../../data/logx/AllT.csv');

# Question 01
The order of the presentation affects the confidence

In [3]:
SELECT ctype, AVG(confidence) AVG_confidence, STDDEV_SAMP(confidence) STD_Deviation_confidence FROM DescEKGCases GROUP BY ctype;

# Question 02
The confidence score (first stage) affects the time it takes to accomplish the second stage.

## Description First

In [4]:
DROP TABLE IF EXISTS Confidence_First_DF;
CREATE VIEW Confidence_First_DF AS
SELECT casen, AVG(confidence) AVG_confidence_first_stage
FROM DescEKGCases
WHERE stage = 'Description Hypothesis' AND ctype='DF'
GROUP BY casen
ORDER by casen;

DROP VIEW IF EXISTS Time_Second_DF;
CREATE VIEW Time_Second_DF AS
SELECT casen, AVG(time) AVG_time_second_stage
FROM DescEKGCases
WHERE stage = 'EKG Hypothesis' AND ctype='DF'
GROUP BY casen
ORDER by casen;

SELECT CF.casen, AVG_confidence_first_stage, AVG_time_second_stage
FROM Confidence_First_DF CF, Time_Second_DF TS
WHERE CF.casen = TS.casen;

## EKG First

In [5]:
DROP VIEW IF EXISTS Confidence_First_EF;
CREATE VIEW Confidence_First_EF AS
SELECT casen, AVG(confidence) AVG_confidence_first_stage
FROM DescEKGCases
WHERE stage = 'EKG Hypothesis' AND ctype='EF'
GROUP BY casen
ORDER by casen;

DROP VIEW IF EXISTS Time_Second_EF;
CREATE VIEW Time_Second_EF AS
SELECT casen, AVG(time) AVG_time_second_stage
FROM DescEKGCases
WHERE stage = 'Description Hypothesis' AND ctype='EF'
GROUP BY casen
ORDER by casen;

SELECT CF.casen, AVG_confidence_first_stage, AVG_time_second_stage
FROM Confidence_First_EF CF, Time_Second_EF TS
WHERE CF.casen = TS.casen;

# Question 3

The confidence score will influence the probability of changing the hypothesis.

## Description First

In [6]:
DROP VIEW IF EXISTS First_Answer_DF;
CREATE VIEW First_Answer_DF AS
SELECT user, casen, answer First_Answer, confidence
FROM DescEKGCases
WHERE stage = 'Description Hypothesis' AND ctype = 'DF';

DROP VIEW IF EXISTS Second_Answer_DF;
CREATE VIEW Second_Answer_DF AS
SELECT user, casen, answer Second_Answer
FROM DescEKGCases
WHERE stage = 'EKG Hypothesis' AND ctype = 'DF';

SELECT D1.user, D1.casen, First_Answer, confidence, Second_Answer
FROM First_Answer_DF D1, Second_Answer_DF D2
WHERE D1.user = D2.user AND D1.casen = D2.casen;

### AVG confidence for same answer

In [7]:
SELECT D1.casen, AVG(confidence) AVG_confidence, COUNT(*)
FROM First_Answer_DF D1, Second_Answer_DF D2
WHERE D1.user = D2.user AND D1.casen = D2.casen AND First_Answer = Second_Answer
GROUP BY D1.casen;

### AVG confidence for different answer

In [8]:
SELECT D1.casen, AVG(confidence) AVG_confidence, COUNT(*)
FROM First_Answer_DF D1, Second_Answer_DF D2
WHERE D1.user = D2.user AND D1.casen = D2.casen AND First_Answer <> Second_Answer
GROUP BY D1.casen;

## EKG First

In [9]:
DROP VIEW IF EXISTS First_Answer_EF;
CREATE VIEW First_Answer_EF AS
SELECT user, casen, answer First_Answer, confidence
FROM DescEKGCases
WHERE stage = 'EKG Hypothesis' AND ctype = 'EF';

DROP VIEW IF EXISTS Second_Answer_EF;
CREATE VIEW Second_Answer_EF AS
SELECT user, casen, answer Second_Answer
FROM DescEKGCases
WHERE stage = 'Description Hypothesis' AND ctype = 'EF';

SELECT E1.user, E1.casen, First_Answer, confidence, Second_Answer
FROM First_Answer_EF E1, Second_Answer_EF E2
WHERE E1.user = E2.user AND E1.casen = E2.casen;

### AVG confidence for same answer

In [10]:
SELECT E1.casen, AVG(confidence) AVG_confidence, COUNT(*)
FROM First_Answer_EF E1, Second_Answer_EF E2
WHERE E1.user = E2.user AND E1.casen = E2.casen AND First_Answer = Second_Answer
GROUP BY E1.casen;

### AVG confidence for different answer

In [11]:
SELECT E1.casen, AVG(confidence) AVG_confidence, COUNT(*)
FROM First_Answer_EF E1, Second_Answer_EF E2
WHERE E1.user = E2.user AND E1.casen = E2.casen AND First_Answer <> Second_Answer
GROUP BY E1.casen;

# Question 4

The time to accomplish the first task between the groups.

In [14]:
DROP VIEW IF EXISTS Confidence_AVG_DF;
CREATE VIEW Confidence_AVG_DF AS
SELECT casen, AVG(time) AVG_Time_Description_First
FROM DescEKGCases
WHERE ctype='DF' AND stage = 'Description Hypothesis'
GROUP BY casen
ORDER BY casen;

DROP VIEW IF EXISTS Confidence_AVG_EF;
CREATE VIEW Confidence_AVG_EF AS
SELECT casen, AVG(time) AVG_Time_EKG_First
FROM DescEKGCases
WHERE ctype='EF' AND stage = 'EKG Hypothesis'
GROUP BY casen
ORDER BY casen;

SELECT DF.casen, AVG_Time_Description_First, AVG_Time_EKG_First
FROM Confidence_AVG_DF DF, Confidence_AVG_EF EF
WHERE DF.casen = EF.casen;

# Question 5

The time between the first and the second task.

## Description First

In [18]:
DROP VIEW IF EXISTS Confidence_AVG_Task1_DF;
CREATE VIEW Confidence_AVG_Task1_DF AS
SELECT casen, AVG(time) AVG_Time_Task1
FROM DescEKGCases
WHERE ctype='DF' AND stage = 'Description Hypothesis'
GROUP BY casen
ORDER BY casen;

DROP VIEW IF EXISTS Confidence_AVG_Task2_DF;
CREATE VIEW Confidence_AVG_Task2_DF AS
SELECT casen, AVG(time) AVG_Time_Task2
FROM DescEKGCases
WHERE ctype='DF' AND stage = 'EKG Hypothesis'
GROUP BY casen
ORDER BY casen;

SELECT T1.casen, AVG_Time_Task1, AVG_Time_Task2
FROM Confidence_AVG_Task1_DF T1, Confidence_AVG_Task2_DF T2
WHERE T1.casen = T2.casen;

## EKG First

In [17]:
DROP VIEW IF EXISTS Confidence_AVG_Task1_EF;
CREATE VIEW Confidence_AVG_Task1_EF AS
SELECT casen, AVG(time) AVG_Time_Task1
FROM DescEKGCases
WHERE ctype='EF' AND stage = 'EKG Hypothesis'
GROUP BY casen
ORDER BY casen;

DROP VIEW IF EXISTS Confidence_AVG_Task2_EF;
CREATE VIEW Confidence_AVG_Task2_EF AS
SELECT casen, AVG(time) AVG_Time_Task2
FROM DescEKGCases
WHERE ctype='EF' AND stage = 'Description Hypothesis'
GROUP BY casen
ORDER BY casen;

SELECT T1.casen, AVG_Time_Task1, AVG_Time_Task2
FROM Confidence_AVG_Task1_EF T1, Confidence_AVG_Task2_EF T2
WHERE T1.casen = T2.casen;

# Question 6

The confidence level in the interpretation of EKG can indicate their proficiency.